In [ ]:
library(sf)
library(terra)
library(leaflet)

In [ ]:
KEN <- st_as_sf(raster::getData("GADM", country="KEN", level=1, path=getwd()), crs = 4326)
polys <- KEN %>% 
  filter(NAME_1 %in% c("Nairobi", "Kiambu", "Machakos", "Murang'a")) %>%
  st_union()

leaflet() %>%
  addTiles() %>% # Add default OpenStreetMap map tiles
  addPolygons(data = polys, weight = 2, fillColor = 'transparent', label = "Central Kenya")

In [ ]:
weather.JSON <- rjson::fromJSON(file = "POWER_Regional_Daily_20170201_20210207_74d3dbce.json")
weather.JSON <- enframe(unlist(weather.JSON))
weather.JSON <- weather.JSON %>% 
  separate(name, into = c(paste0("name", 1:5)), fill = "right") %>%
  filter(name1 == "features" &
           name2 == "geometry" &
           name3 %in% c("coordinates1", "coordinates2") |
           (name1 == "features" &
              name2 == "properties" &
              name3 == "parameter")) %>%
  select(name5, value)